<a href="https://colab.research.google.com/github/jessiececilya/Projects/blob/main/Ensemble_WIDS_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score  #for accuracy_score
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.metrics import confusion_matrix #for confusion matrix

In [ ]:
train= pd.read_csv("https://raw.githubusercontent.com/jessiececilya/ML-Data/main/WiDS/TrainingWiDS2021.csv")

In [ ]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [ ]:
train.drop(['Unnamed: 0','encounter_id','hospital_id','icu_id'],axis=1,inplace=True)

In [ ]:
dd= pd.read_csv("https://raw.githubusercontent.com/jessiececilya/ML-Data/main/WiDS/DataDictionaryWiDS2021.csv")

In [ ]:
bincols= (dd.loc[dd['Data Type']=='binary','Variable Name']).to_list()

In [ ]:
num= dd.loc[(dd['Data Type']=='integer') | (dd['Data Type']=='numeric' ),'Variable Name']
numcols= num.to_list()+['bmi','apache_2_diagnosis','apache_3j_diagnosis']

In [ ]:
numcols.remove('hospital_id')
numcols.remove('encounter_id')
numcols.remove('icu_id')

In [ ]:
cate= ['ethnicity',
 'gender',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_stay_type',
 'icu_type']

In [ ]:
#https://machinelearningmastery.com/statistical-imputation-for-missing-values-in-machine-learning/
#Missing Value imputation

In [ ]:
for element in bincols:
    train[''+element]=train[''+element].fillna(train[''+element].mode()[0])

for element in numcols:
    train[''+element]=train[''+element].fillna(round(np.mean(train[''+element]),2))

In [ ]:
#trainx= pd.get_dummies(trainx, columns= cate)
#Label encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for ob in cate:
    trainx[ob] = le.fit_transform(trainx[ob].astype(str))
#predict[ob] = le.fit_transform(predict[ob].astype(str))

TypeError: ignored

In [ ]:
trainx= train.drop(['diabetes_mellitus'],axis=1)
trainy=train['diabetes_mellitus']

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
modelg = GradientBoostingClassifier(random_state=42)
param_grid = {"loss": ["deviance"],
              'n_estimators' : [1000],
              'learning_rate': [0.1],
              'min_samples_leaf': [500],
              }

modelgbm = GridSearchCV(modelg,param_grid = param_grid, cv=2, scoring="accuracy", n_jobs= -1, verbose = 1)
x=modelgbm.fit(trainx,trainy)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 28.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 28.1min finished


In [ ]:

#XGBoost

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
modelx = XGBClassifier()
params = {
    "learning_rate" :[0.1],
    "max_depth" : [4],
    "n_estimators" : [1000]
}

modelxbm = GridSearchCV(modelx,params, cv=2, scoring="accuracy", n_jobs= -1, verbose = 1)

y=modelxbm.fit(trainx,trainy)


Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  8.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  8.6min finished


In [ ]:
#Adaboost

from sklearn.ensemble import AdaBoostClassifier

modela =AdaBoostClassifier(random_state=42)

parameters= {
    "learning_rate" :[0.5,1],
    "n_estimators" : [100] 
}
modelabm= GridSearchCV(modela,parameters, cv=2, scoring="accuracy", n_jobs= -1, verbose = 1)
z=modelabm.fit(trainx,trainy)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  2.3min finished


In [ ]:
#https://stackabuse.com/ensemble-voting-classification-in-python-with-scikit-learn/
# Voting Classifier
from sklearn.ensemble import VotingClassifier
voting_clf= VotingClassifier(estimators=[('GBM',modelgbm),('XBM',modelxbm),('ABM',modelabm)],voting='hard')
a=voting_clf.fit(trainx,trainy)




Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 27.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 27.9min finished


Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  8.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  8.6min finished


Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  2.2min finished


In [ ]:
#Predict
gbm_pred = x.predict_proba(pred)
xbm_pred = y.predict_proba(pred[trainx.columns])
abm_pred = z.predict_proba(pred)
#vc = a.predict_proba(pred)




In [ ]:
averaged_preds = (gbm_pred[:,1]+xbm_pred[:,1]+abm_pred[:,1])/3

In [ ]:
 len(trainx.columns)
 

176

In [ ]:
predict= pd.read_csv('https://raw.githubusercontent.com/jessiececilya/ML-Data/main/WiDS/UnlabeledWiDS2021.csv')

In [ ]:
for element in bincols:
    predict[''+element]=predict[''+element].fillna(predict[''+element].mode()[0])

for element in numcols:
    predict[''+element]=predict[''+element].fillna(round(np.mean(predict[''+element]),2))

In [ ]:
pred= predict.drop(['Unnamed: 0','encounter_id','hospital_id','icu_id'],axis=1)

In [ ]:
pred.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,72,29.12,0,Caucasian,F,152.4,Floor,Accident & Emergency,admit,Med-Surg ICU,0.015278,0,83.48,2.80,110.0,104.01,0,0,1.90,44.00,1.49,0.61,4.0,6.0,0.0,5.0,97.0,38.0,39.90,0,54.0,41.17,41.17,136.85,7.36,31.0,130.00,36.4,1943.53,0,5.40,80.12,47.41,104.0,40.0,104.0,40.0,66.0,38.0,115.54,61.87,123.0,54.0,123.0,54.0,35.0,15.0,100.0,96.0,153.03,93.56,149.0,76.0,149.0,76.0,38.7,35.6,68.89,57.17,59.00,59.00,59.00,59.00,46.0,44.0,94.03,75.81,80.00,80.00,80.00,80.00,19.0,19.0,97.0,96.0,138.17,115.16,116.0,116.00,116.00,116.00,36.4,36.4,2.80,2.80,1.90,1.9,44.0,40.0,9.8,9.3,1.49,1.44,104.0,97.0,26.0,23.0,14.50,14.20,42.80,39.90,1.59,1.45,2.95,2.13,173.00,173.00,5.7,4.9,132.0,130.0,5.60,5.4,2.80,2.80,1.90,1.90,40.00,40.00,9.80,9.80,1.49,1.49,104.00,104.00,26.00,26.00,14.50,14.50,42.80,42.80,1.59,1.45,2.97,2.91,173.00,173.00,4.90,4.90,132.00,132.00,5.60,5.6,44.67,37.77,7.39,7.33,173.48,103.11,297.8,224.42,43.36,42.2,7.35,7.33,176.3,156.77,254.69,246.26,0,0,0,0,0,0,0
1,86,29.12,0,Caucasian,F,175.3,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,0.000000,0,83.48,2.87,117.0,106.01,0,0,1.15,19.00,0.92,0.61,4.0,6.0,0.0,5.0,73.0,116.0,32.89,0,41.0

In [ ]:
le = LabelEncoder()
for ob in cate:
  pred[ob] = le.fit_transform(pred[ob].astype(str))

In [ ]:
pred.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,72,29.12,0,2,0,152.4,3,0,0,5,0.015278,0,83.48,2.80,110.0,104.01,0,0,1.90,44.00,1.49,0.61,4.0,6.0,0.0,5.0,97.0,38.0,39.90,0,54.0,41.17,41.17,136.85,7.36,31.0,130.00,36.4,1943.53,0,5.40,80.12,47.41,104.0,40.0,104.0,40.0,66.0,38.0,115.54,61.87,123.0,54.0,123.0,54.0,35.0,15.0,100.0,96.0,153.03,93.56,149.0,76.0,149.0,76.0,38.7,35.6,68.89,57.17,59.00,59.00,59.00,59.00,46.0,44.0,94.03,75.81,80.00,80.00,80.00,80.00,19.0,19.0,97.0,96.0,138.17,115.16,116.0,116.00,116.00,116.00,36.4,36.4,2.80,2.80,1.90,1.9,44.0,40.0,9.8,9.3,1.49,1.44,104.0,97.0,26.0,23.0,14.50,14.20,42.80,39.90,1.59,1.45,2.95,2.13,173.00,173.00,5.7,4.9,132.0,130.0,5.60,5.4,2.80,2.80,1.90,1.90,40.00,40.00,9.80,9.80,1.49,1.49,104.00,104.00,26.00,26.00,14.50,14.50,42.80,42.80,1.59,1.45,2.97,2.91,173.00,173.00,4.90,4.90,132.00,132.00,5.60,5.6,44.67,37.77,7.39,7.33,173.48,103.11,297.8,224.42,43.36,42.2,7.35,7.33,176.3,156.77,254.69,246.26,0,0,0,0,0,0,0
1,86,29.12,0,2,0,175.3,2,0,0,5,0.000000,0,83.48,2.87,117.0,106.01,0,0,1.15,19.00,0.92,0.61,4.0,6.0,0.0,5.0,73.0,116.0,32.89,0,41.0,41.17,41.17,136.85,7.36,53.0,142.00,36.3,1943.53,0,12.02,80.12,47.41,101.0,27.0,101.0,27.0,116.0,56.0,115.

In [ ]:
#pred= pd.get_dummies(pred, columns= cate)

In [ ]:
 set(pred.columns) -set(trainx.columns)

set()

In [ ]:
# addcols= ['hospital_admit_source_Acute Care/Floor', 'hospital_admit_source_ICU', 'hospital_admit_source_Observation', 'hospital_admit_source_Other', 'hospital_admit_source_PACU']
# for e in addcols:
#   pred[''+e]=0

In [ ]:
encounterid= predict['encounter_id']

In [ ]:
output= averaged_preds

In [ ]:
out= pd.DataFrame({
    'encounter_id' : encounterid.tolist(),
    'diabetes_mellitus' : output.tolist()
    
})

In [ ]:
out.sort_values('encounter_id', ascending=True,inplace=True)

In [ ]:
from google.colab import files
out.to_csv('avg.csv',index=False)
files.download('avg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
output

array([0.04937827, 0.07608976, 0.14540071, ..., 0.15342361, 0.02029547,
       0.02526344])